In [3]:
import os
import pandas as pd
import glob

from pathlib import Path

In [4]:
dir_path = Path(os.getcwd())
data_dir = str(os.path.join(dir_path.parent,'data'))
balance_files = glob.glob(data_dir + "/poker_balances*.csv")
hands_files = glob.glob(data_dir + "/poker_hands*.csv")
table_files = glob.glob(data_dir + "/poker_table_info*.csv")

def get_hands_df():
    hands_dfs=[]

    for file in hands_files:
        new_df = pd.read_csv(file)
        hands_dfs.append(new_df)

    hands = pd.concat(hands_dfs)
    return hands
    
def get_games_df():
    balance_dfs=[]
    
    for file in balance_files:
        new_df = pd.read_csv(file)
        balance_dfs.append(new_df)
        
    balances = pd.concat(balance_dfs)
    return balances

def get_table_df():
    table_dfs=[]
    
    for file in table_files:
        new_df = pd.read_csv(file)
        table_dfs.append(new_df)
        
    tables = pd.concat(table_dfs)
    return tables


In [5]:
pd.set_option('display.max_columns', None)  
hands_base = get_hands_df()
games_base = get_games_df()
tables = get_table_df()

games = pd.merge(games_base,tables)
hands = pd.merge(hands_base,games)

In [4]:
tables

,table_id,scenario_name,player_types
0,1,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,10,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,11,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,12,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,13,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,14,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,15,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,16,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,17,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...
0,18,all_call_against_smart_player,AlwaysCallPlayer|AlwaysCallPlayer|AlwaysCallPl...


In [6]:
games

,table_id,game_id,player_name,player_type,game_result,game_reason,blind_type,final_hand,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types
0,1,1,players_1,AlwaysCallPlayer,lost,lost_game,small,one_pair,100000,100000.0,99990.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
1,1,2,players_1,AlwaysCallPlayer,won,last_man_standing,big,one_pair,100000,99990.0,100000.0,10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
2,1,3,players_1,AlwaysCallPlayer,won,won_game,small,straight,100000,100000.0,100010.0,10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
3,1,4,players_1,AlwaysCallPlayer,won,won_game,big,two_pair,100000,100010.0,100020.0,10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
4,1,5,players_1,AlwaysCallPlayer,won,won_game,small,one_pair,100000,100020.0,100030.0,10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
25995,9,896,players_2,SmartPlayer,lost,fold,small,full_house,100000,111460.0,111450.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
25996,9,897,players_2,SmartPlayer,lost,fold,big,full_house,100000,111450.0,111440.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
25997,9,898,players_2,SmartPlayer,lost,lost_game,small,two_pair,100000,111440.0,111430.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
25998,9,899,players_2,SmartPlayer,lost,fold,big,two_pair,100000,111430.0,111420.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer


In [7]:
hands

,table_id,game_id,player_name,player_type,bet_number,opponents,call,current,final,pot,allowed,hand1,hand2,community1,community2,community3,community4,community5,game_result,game_reason,blind_type,final_hand,beginning_balance,game_start_balance,game_end_balance,game_net_change,scenario_name,player_types
0,1,1,players_1,AlwaysCallPlayer,1,1,5.0,5.0,10.0,15.0,True,4-diamonds,Q-diamonds,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,lost,lost_game,small,one_pair,100000,100000.0,99990.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
1,1,1,players_1,AlwaysCallPlayer,2,1,0.0,10.0,10.0,20.0,True,4-diamonds,Q-diamonds,5-clubs,6-hearts,8-spades,Z-N/A,Z-N/A,lost,lost_game,small,one_pair,100000,100000.0,99990.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
2,1,1,players_1,AlwaysCallPlayer,3,1,0.0,10.0,10.0,20.0,True,4-diamonds,Q-diamonds,5-clubs,6-hearts,8-spades,K-spades,Z-N/A,lost,lost_game,small,one_pair,100000,100000.0,99990.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
3,1,1,players_1,AlwaysCallPlayer,4,1,0.0,10.0,10.0,20.0,True,4-diamonds,Q-diamonds,5-clubs,6-hearts,8-spades,K-hearts,K-spades,lost,lost_game,small,one_pair,100000,100000.0,99990.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
4,1,2,players_1,AlwaysCallPlayer,1,1,0.0,10.0,10.0,20.0,True,J-spades,Q-hearts,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,won,last_man_standing,big,one_pair,100000,99990.0,100000.0,10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
137648,9,898,players_2,SmartPlayer,4,1,0.0,10.0,10.0,20.0,True,2-hearts,4-spades,2-clubs,7-clubs,A-clubs,J-hearts,J-spades,lost,lost_game,small,two_pair,100000,111440.0,111430.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
137649,9,899,players_2,SmartPlayer,1,1,0.0,10.0,10.0,20.0,True,10-clubs,5-diamonds,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,lost,fold,big,two_pair,100000,111430.0,111420.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
137650,9,899,players_2,SmartPlayer,2,1,0.0,10.0,NaN,20.0,True,10-clubs,5-diamonds,2-spades,7-hearts,K-clubs,Z-N/A,Z-N/A,lost,fold,big,two_pair,100000,111430.0,111420.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer
137651,9,900,players_2,SmartPlayer,1,1,5.0,5.0,10.0,15.0,True,5-diamonds,7-spades,Z-N/A,Z-N/A,Z-N/A,Z-N/A,Z-N/A,lost,fold,small,two_pair,100000,111420.0,111410.0,-10.0,smart vs 1 all call player,AlwaysCallPlayer|SmartPlayer


# Basic Game Statistics

Number of Games played in the simulation

In [7]:
games = games[games['scenario_name'] == 'all_raise_against_smart_player']

In [8]:
number_of_games = len(set(games.game_id))
number_of_games

2500

Players Mean number of chips gained and variance

In [9]:
chips_gained_per_game = games.groupby('player_name').mean('game_net_change')[['game_net_change']].reset_index()
variance_calculation = games[['player_name','game_net_change']]
variance_calculation['game_net_change'] = variance_calculation['game_net_change'] ** 2
expectation_squared = variance_calculation.groupby('player_name').mean('game_net_change').reset_index()
merged_expectations = pd.merge(expectation_squared,chips_gained_per_game,on='player_name')
merged_expectations['variance'] = merged_expectations['game_net_change_x'] - merged_expectations['game_net_change_y']
summary_stats = merged_expectations[['player_name','game_net_change_y','variance']]
summary_stats.columns = ['player_name','mean','variance']
summary_stats['games'] = number_of_games
summary_stats

c:\users\silbe\appdata\local\programs\python\python37-32\lib\site-packages\ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


,player_name,mean,variance,games
0,players_1,0.5108,3.5088,2500
1,players_2,0.5108,3.5088,2500
2,players_3,0.5108,3.5088,2500
3,players_4,0.5108,3.5088,2500
4,players_5,0.5108,3.5088,2500
5,players_6,-2.5540,103.0440,2500


Top 10 Best Card Hands Dealt by Mean Number of Chips Gained 

In [10]:
two_card_hands = hands[['game_id','player_name','player_type','hand1','hand2','game_net_change']]
two_card_hands = two_card_hands.drop_duplicates()
best_two_card_hands = two_card_hands.groupby(['player_name','hand1','hand2']).mean().reset_index()[['player_name','hand1','hand2','game_net_change']]
best_two_card_hands = best_two_card_hands.sort_values(by=['player_name','game_net_change'], ascending=[True,False])
best_two_card_hands.groupby('player_name').head(10)

,player_name,hand1,hand2,game_net_change
830,players_1,6-clubs,J-clubs,12.000000
186,players_1,10-spades,K-hearts,10.000000
356,players_1,2-spades,9-spades,9.000000
1095,players_1,8-spades,9-hearts,8.666667
1094,players_1,8-spades,9-diamonds,8.000000
1234,players_1,J-clubs,K-clubs,8.000000
401,players_1,3-clubs,A-hearts,7.000000
520,players_1,3-spades,9-spades,7.000000
120,players_1,10-hearts,7-diamonds,6.000000
299,players_1,2-hearts,6-hearts,6.000000
